# SPOC Code Generation (Colab GPU)

Generate code on GPU, then download results for local evaluation.

## Setup

In [15]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Clone repo (first time only)
!git clone https://github.com/tlebryk/spoc.git
# %cd YOUR_REPO/spoc

fatal: destination path 'spoc' already exists and is not an empty directory.


In [17]:
# Install dependencies
!pip install -q transformers torch accelerate pandas tqdm sacrebleu azure-storage-blob

In [18]:
# Download SPOC data (if not in repo)
!wget https://sumith1896.github.io/spoc/data/spoc.zip
!unzip -q spoc.zip && mv spoc/* . && rm -rf spoc spoc.zip

--2025-11-25 04:30:33--  https://sumith1896.github.io/spoc/data/spoc.zip
Resolving sumith1896.github.io (sumith1896.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to sumith1896.github.io (sumith1896.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9960355 (9.5M) [application/x-zip-compressed]
Saving to: ‘spoc.zip’

spoc.zip            100%[===================>]   9.50M  --.-KB/s    in 0.07s   

2025-11-25 04:30:33 (145 MB/s) - ‘spoc.zip’ saved [9960355/9960355]

replace test/spoc-testw.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Load Modules

In [ ]:
import os
from google.colab import userdata

os.environ["AZURE_STORAGE_CONNECTION_STRING"] = userdata.get("AZURE_STORAGE_CONNECTION_STRING")

In [11]:
import sys
sys.path.insert(0, './src')

from data_loader import SPOCDataLoader
from inference import CodeGenerator

ModuleNotFoundError: No module named 'data_loader'

## Generate Code (GPU)

In [21]:
!python run_eval.py --n-samples 2


SPOC Code Generation Evaluation

Model: Qwen/Qwen2.5-Coder-0.5B
Split: testp
N samples: 2
Temperature: 0.2
Using hidden test cases

⚠ Azure not configured (local only)

--------------------------------------------------------------------------------
Loading data...
--------------------------------------------------------------------------------
Traceback (most recent call last):
  File "/content/scripts/run_eval.py", line 282, in <module>
  File "/content/scripts/run_eval.py", line 140, in main
    programs = loader.load_test_split(args.split)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/src/data_loader.py", line 126, in load_test_split
    return self.group_into_programs(df)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/src/data_loader.py", line 101, in group_into_programs
    group = group.sort_values('line')
            ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/core/frame.py", line 7209, in sort_values
    if is

In [22]:
!ls outputs

In [14]:
%cd src

/content/src


In [17]:
"""Example usage of Azure Blob Storage."""

import sys
from pathlib import Path
# sys.path.insert(0, str(Path(__file__).parent.parent / "src"))

from azure_storage import AzureStorage

# Initialize (reads AZURE_STORAGE_CONNECTION_STRING from environment)
storage = AzureStorage(container="spoc")

# Save evaluation results
results = {
    "timestamp": "20250124_120000",
    "metrics": {"CompAcc": 0.85, "FCorrAcc": 0.72},
    "programs": ["program1", "program2"]
}
# storage.save_json(results, "outputs/eval_20250124.json")

# Load results

loaded = storage.load_json("outputs/eval_20250124.json")

# Save a file
storage.save_file("outputs/metrics.txt", "outputs/metrics_20250124.txt")

# Download a file
storage.load_file("outputs/metrics_20250124.txt", "local_metrics.txt")

# List all evaluation results
eval_files = storage.list(prefix="outputs/eval_")
print(f"Found {len(eval_files)} evaluation results")


FileNotFoundError: [Errno 2] No such file or directory: 'outputs/metrics.txt'

In [1]:
!pip install azure-storage-blob

In [2]:
from azure.storage.blob import BlobServiceClient



In [18]:
loaded

{'timestamp': '20250124_120000',
 'metrics': {'CompAcc': 0.85, 'FCorrAcc': 0.72},
 'programs': ['program1', 'program2']}